# Linear regression 

In this exercise you will use linear regression to predict flat prices. One more time, training will be handled via gradient descent. Although, contratry to the first exercise, we will now:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a different loss function.

Let's start with getting the data.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2017-11-03 17:46:34--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Translacja www.dropbox.com (www.dropbox.com)... 162.125.66.1, 2620:100:6022:1::a27d:4201
Łączenie się z www.dropbox.com (www.dropbox.com)|162.125.66.1|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 302 Found
Lokalizacja: https://dl.dropboxusercontent.com/content_link/vjqxAGjNeOPzHheSu5428fpWuGDnJ8OvW6g30L8VfNXI2FDVT8HkeGCfHQNd98pY/file?dl=1 [podążanie]
--2017-11-03 17:46:35--  https://dl.dropboxusercontent.com/content_link/vjqxAGjNeOPzHheSu5428fpWuGDnJ8OvW6g30L8VfNXI2FDVT8HkeGCfHQNd98pY/file?dl=1
Translacja dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.66.6, 2620:100:6022:6::a27d:4206
Łączenie się z dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.66.6|:443... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 200 OK
Długość: 6211 (6,1K) [application/binary]
Zapis do: `mieszkania.csv'

mieszkania.csv      100%[===================>]   6,0

In [2]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny`). 

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [3]:
import math

def msle(ys, ps):
    """
    Mean squared logarithmic error.
    :param ys: ground truth prices
    :param ps: prediction prices
    """
    assert len(ys) == len(ps)
    return sum((math.log(1 + y_i) - math.log(1 + p_i)) ** 2 for y_i, p_i in zip(ys, ps)) / len(ys)

def l2_loss(ys, ps):
    """
    Least square error.
    :param ys: ground truth prices
    :param ps: prediction prices
    """
    # quicker solution
    # return np.linalg.norm(y - x) / len(ys)
    return math.sqrt(sum((y - x) ** 2 for x, y in zip(ys, ps)) / len(ys))

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [31]:
import pandas
import numpy as np
import scipy.stats.mstats


def get_dataset(path):
    with open(path) as flats:
        data = pandas.read_csv(flats)
    #data = data.sample(frac=1)
    return data

def get_training_dataset():
    data_path = 'mieszkania.csv'
    return get_dataset(data_path)

dataset = get_training_dataset()

def get_testing_dataset():
    data_path = 'mieszkania_test.csv'
    return get_dataset(data_path)

def get_mean_price():
    return np.mean(dataset['cena'])

ys = dataset['cena']
ps = [get_mean_price()] * len(ys)

print(msle(ys, ps))
print(l2_loss(ys, ps))

0.3915253538257009
271680.69772714784


Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [32]:
def get_geometric_mean_price():
    return scipy.stats.mstats.gmean(dataset['cena'])

ps = [get_geometric_mean_price()] * len(ys)

print(msle(ys, ps))
print(l2_loss(ys, ps))

0.36488961221361227
282241.4511487305


In [33]:
def get_districts_set():
    return set(dataset['dzielnica'])

print(get_districts_set())

{'grodziskowo', 'mokotowo', 'ochotowo', 'wolowo'}


Now, let's implement a standard linear regression model. 

Let's make some features

In [90]:
lr = 10 # step size
n_epoch = 1000 # number of passes over training data
features_number = 5

# initial_weights
weights = np.zeros(features_number + 1)

data = dataset    

m2 = [item / 200 for item in data['m2']]
bedrooms = data['ilość_sypialni']
bathrooms = data['ilość_łazienek']
construction_year = [(2017 - year) / 100 for year in data['rok_budowy']]
parking_lot = data['parking_podziemny']

xs = np.array([m2, bedrooms, bathrooms, construction_year, parking_lot]).T
prices = dataset['cena']
print(xs.shape)

(200, 5)


In [92]:
def predict_one(w, x):
    return x.dot(w[:-1]) + w[-1]

def predict(w, xs=xs):
    results = list()
    for item in xs:
        results.append(predict_one(w, item))
    return results

def evaluate(w, xs=xs, ys=ys, evaluation_function=msle):
    return msle(ys, predict(w, xs))

losses = list()

def derivatives_one_point(ind, w):
    x = xs[ind]
    prediction = predict_one(w, x)
    #print(prediction, prices[ind])
    const_factor = -2 * (math.log(1 + prices[ind]) - math.log(1 + prediction)) / (1 + prediction)
    #print(const_factor)
    #const_factor = 2 * (prediction - prices[ind])
    result = np.append(np.array(x), [1]) * const_factor
    return result

for i in range(n_epoch):
    loss = evaluate(weights)
    losses.append(loss)
    derivatives = np.array(sum(derivatives_one_point(ind, weights) for ind in range(len(xs))))
    weights = weights - lr * derivatives
    print('Iter: {:>3} Loss: {:8.8f}'.format(i, loss))


Iter:   0 Loss: 0.13873438
Iter:   1 Loss: 0.13873438
Iter:   2 Loss: 0.13873438
Iter:   3 Loss: 0.13873438
Iter:   4 Loss: 0.13873438
Iter:   5 Loss: 0.13873438
Iter:   6 Loss: 0.13873438
Iter:   7 Loss: 0.13873438
Iter:   8 Loss: 0.13873437
Iter:   9 Loss: 0.13873437
Iter:  10 Loss: 0.13873437
Iter:  11 Loss: 0.13873437
Iter:  12 Loss: 0.13873437
Iter:  13 Loss: 0.13873437
Iter:  14 Loss: 0.13873437
Iter:  15 Loss: 0.13873437
Iter:  16 Loss: 0.13873437
Iter:  17 Loss: 0.13873437
Iter:  18 Loss: 0.13873436
Iter:  19 Loss: 0.13873436
Iter:  20 Loss: 0.13873436
Iter:  21 Loss: 0.13873436
Iter:  22 Loss: 0.13873436
Iter:  23 Loss: 0.13873436
Iter:  24 Loss: 0.13873436
Iter:  25 Loss: 0.13873436
Iter:  26 Loss: 0.13873436
Iter:  27 Loss: 0.13873436
Iter:  28 Loss: 0.13873435
Iter:  29 Loss: 0.13873435
Iter:  30 Loss: 0.13873435
Iter:  31 Loss: 0.13873435
Iter:  32 Loss: 0.13873435
Iter:  33 Loss: 0.13873435
Iter:  34 Loss: 0.13873435
Iter:  35 Loss: 0.13873435
Iter:  36 Loss: 0.13873435
I

Iter: 326 Loss: 0.13873404
Iter: 327 Loss: 0.13873404
Iter: 328 Loss: 0.13873404
Iter: 329 Loss: 0.13873404
Iter: 330 Loss: 0.13873404
Iter: 331 Loss: 0.13873404
Iter: 332 Loss: 0.13873404
Iter: 333 Loss: 0.13873404
Iter: 334 Loss: 0.13873404
Iter: 335 Loss: 0.13873403
Iter: 336 Loss: 0.13873403
Iter: 337 Loss: 0.13873403
Iter: 338 Loss: 0.13873403
Iter: 339 Loss: 0.13873403
Iter: 340 Loss: 0.13873403
Iter: 341 Loss: 0.13873403
Iter: 342 Loss: 0.13873403
Iter: 343 Loss: 0.13873403
Iter: 344 Loss: 0.13873403
Iter: 345 Loss: 0.13873402
Iter: 346 Loss: 0.13873402
Iter: 347 Loss: 0.13873402
Iter: 348 Loss: 0.13873402
Iter: 349 Loss: 0.13873402
Iter: 350 Loss: 0.13873402
Iter: 351 Loss: 0.13873402
Iter: 352 Loss: 0.13873402
Iter: 353 Loss: 0.13873402
Iter: 354 Loss: 0.13873401
Iter: 355 Loss: 0.13873401
Iter: 356 Loss: 0.13873401
Iter: 357 Loss: 0.13873401
Iter: 358 Loss: 0.13873401
Iter: 359 Loss: 0.13873401
Iter: 360 Loss: 0.13873401
Iter: 361 Loss: 0.13873401
Iter: 362 Loss: 0.13873401
I

Iter: 633 Loss: 0.13873372
Iter: 634 Loss: 0.13873372
Iter: 635 Loss: 0.13873372
Iter: 636 Loss: 0.13873372
Iter: 637 Loss: 0.13873372
Iter: 638 Loss: 0.13873372
Iter: 639 Loss: 0.13873372
Iter: 640 Loss: 0.13873372
Iter: 641 Loss: 0.13873372
Iter: 642 Loss: 0.13873371
Iter: 643 Loss: 0.13873371
Iter: 644 Loss: 0.13873371
Iter: 645 Loss: 0.13873371
Iter: 646 Loss: 0.13873371
Iter: 647 Loss: 0.13873371
Iter: 648 Loss: 0.13873371
Iter: 649 Loss: 0.13873371
Iter: 650 Loss: 0.13873371
Iter: 651 Loss: 0.13873371
Iter: 652 Loss: 0.13873370
Iter: 653 Loss: 0.13873370
Iter: 654 Loss: 0.13873370
Iter: 655 Loss: 0.13873370
Iter: 656 Loss: 0.13873370
Iter: 657 Loss: 0.13873370
Iter: 658 Loss: 0.13873370
Iter: 659 Loss: 0.13873370
Iter: 660 Loss: 0.13873370
Iter: 661 Loss: 0.13873369
Iter: 662 Loss: 0.13873369
Iter: 663 Loss: 0.13873369
Iter: 664 Loss: 0.13873369
Iter: 665 Loss: 0.13873369
Iter: 666 Loss: 0.13873369
Iter: 667 Loss: 0.13873369
Iter: 668 Loss: 0.13873369
Iter: 669 Loss: 0.13873369
I

Iter: 937 Loss: 0.13873341
Iter: 938 Loss: 0.13873341
Iter: 939 Loss: 0.13873341
Iter: 940 Loss: 0.13873340
Iter: 941 Loss: 0.13873340
Iter: 942 Loss: 0.13873340
Iter: 943 Loss: 0.13873340
Iter: 944 Loss: 0.13873340
Iter: 945 Loss: 0.13873340
Iter: 946 Loss: 0.13873340
Iter: 947 Loss: 0.13873340
Iter: 948 Loss: 0.13873340
Iter: 949 Loss: 0.13873339
Iter: 950 Loss: 0.13873339
Iter: 951 Loss: 0.13873339
Iter: 952 Loss: 0.13873339
Iter: 953 Loss: 0.13873339
Iter: 954 Loss: 0.13873339
Iter: 955 Loss: 0.13873339
Iter: 956 Loss: 0.13873339
Iter: 957 Loss: 0.13873339
Iter: 958 Loss: 0.13873339
Iter: 959 Loss: 0.13873338
Iter: 960 Loss: 0.13873338
Iter: 961 Loss: 0.13873338
Iter: 962 Loss: 0.13873338
Iter: 963 Loss: 0.13873338
Iter: 964 Loss: 0.13873338
Iter: 965 Loss: 0.13873338
Iter: 966 Loss: 0.13873338
Iter: 967 Loss: 0.13873338
Iter: 968 Loss: 0.13873338
Iter: 969 Loss: 0.13873337
Iter: 970 Loss: 0.13873337
Iter: 971 Loss: 0.13873337
Iter: 972 Loss: 0.13873337
Iter: 973 Loss: 0.13873337
I

In [67]:
from sklearn.linear_model import LinearRegression
import numpy as np

X = xs
regr = LinearRegression()
regr.fit(X, ys) # training

sk_loss = l2_loss(ys, regr.predict(X))
print(regr.coef_, regr.intercept_)
print(sk_loss)

weights = np.array(list(regr.coef_) + [regr.intercept_])

[ 1152488.53982151    17666.40430621    12459.83686168   -64202.34938801
    28010.49825305] 12796.0122387
119041.4487204289


In [50]:
for price, prediction in zip(ys, predict(weights, xs)):
    print(price, prediction)

780094 188.111489168
346912 110.64307325
523466 231.17731794
830965 230.076152587
1090479 246.614079585
599060 177.254826049
463639 145.95931201
166785 104.511848236
318849 99.8786044922
1011395 209.851424964
429462 146.505883275
1051608 213.261869284
563473 191.234607917
620449 208.672751061
1097777 253.425544168
645200 186.098918428
776122 179.624208089
367862 139.654263181
257190 115.446019905
1010002 246.905416567
381594 175.116609827
980560 220.236373791
291748 144.834078191
788216 177.316288956
1064711 256.149085931
758000 177.222734762
652028 174.386941584
515061 212.052463928
505024 152.385885417
479861 231.344369274
550903 242.412504114
112635 96.0231659231
291166 115.240214296
128308 114.085608858
629152 226.390416929
482861 172.24471395
330775 104.006682823
453511 178.588517054
232030 100.216718572
392325 139.546023177
306354 147.140596089
922621 225.493697351
452189 177.371088875
994035 223.755058115
488394 152.913759462
384485 110.312981993
105554 105.885653351
496032 150.

### TODO
* Perform normalization/scaling of the data.
* Scikit-learn implementation
* Permutation of data
* !!! separation of testing and training data
* To obtain a feature from the district make it binary (yes/no)

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint: 
<sub><sup><sub><sup><sub><sup>
Be lazy. We don't want to change algorithm.
</sup></sub></sup></sub></sup></sub>

In [ ]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code. 

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint: 
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2: 
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [ ]:
###############################################
# TODO: Implement the feature engieering part #
###############################################

In [ ]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################